# What-if Analysis with RTS/TTS filling options

This notebook describes how to perform what-if analysis using two different missing value filling options. As a sample use-case, we use product demand and price data with some missing values, and see how product demand forecast changes if we increase the prices of products.

Following is the steps:

1. [Import libraries and setup AWS resources](#Import-libraries-and-setup-AWS-resources)
2. [Prepare training dataset CSVs](#Prepare-training-dataset-CSVs)
3. [Create DatasetGroup and Datasets](#Create-DatasetGroup-and-Datasets)
4. [Import the target time series data, and related time series data](#Import-the-target-time-series-data,-and-related-time-series-data)
5. [Create the first Predictor](#Create-the-first-Predictor)
6. [Create Forecast from the first Predictor](#Create-Forecast-from-the-first-Predictor)
7. [Create 2nd Predictor with different futurefill option](#Create-2nd-Predictor-with-different-futurefill-option)
8. [Create Forecast from the 2nd predictor](#Create-Forecast-from-the-2nd-predictor)
9. [Query forecasts, visualize and compare](#Query-forecasts,-visualize-and-compare)
10. [Resource Cleanup](#Resource-cleanup)

**Note** : In order to get two versions of forecast with different filling options, this notebook is creating two Predictors, but please note that it is also possible to perform what-if analysis with just one Predictor with multiple imports of related time series dataset.

## Import libraries and setup AWS resources

In [ ]:
import sys
import os
import time
import datetime

import pandas as pd
import matplotlib.pyplot as plt

import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucket_name", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucket_name = text_widget_bucket.value
assert bucket_name, "bucket_name not set."

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region)
s3 = session.client('s3')
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-WhatIfAnalysis"
role_arn = util.get_or_create_iam_role( role_name = role_name )

## Prepare training dataset CSVs
    
1. Load historical product demand data
2. Check the loaded data, and confirm missing values
3. Split into target time series (demand) and related time series (price)
4. Upload them onto S3

In [ ]:
df = pd.read_csv( "./data/product_demand_with_nan.csv" )
df

In [ ]:
# Try to visualize TTS/RTS.
# You can see gap in the lines (missing values)
df[ df["item_id"]=="item_001" ].plot( x="timestamp" )

In [ ]:
df[ df["item_id"]=="item_001" ]

In [ ]:
# check how many missing values exist in the data
df.isna().sum()

In [ ]:
df_tts = df[["item_id", "timestamp", "demand" ]]
df_rts = df[["item_id", "timestamp", "price" ]]

In [ ]:
df_tts.to_csv( "./data/tts.csv", index=False )
df_rts.to_csv( "./data/rts.csv", index=False )

#### Upload to S3

In [ ]:
project = "whatif_and_filling"

key_tts = "%s/tts.csv" % project
key_rts = "%s/rts.csv" % project

s3.upload_file( Filename="./data/tts.csv", Bucket=bucket_name, Key=key_tts )
s3.upload_file( Filename="./data/rts.csv", Bucket=bucket_name, Key=key_rts )

s3_data_path_tts = "s3://" + bucket_name + "/" + key_tts
s3_data_path_rts = "s3://" + bucket_name + "/" + key_rts

# Create DatasetGroup and Datasets
    
Creating single set of DatasetGroup, Datasets. Please note that we don't have to create two RELATED_TIME_SERIES datasets.

In [ ]:
response = forecast.create_dataset_group(
    DatasetGroupName = project + "_dsg",
    Domain="RETAIL",
    )

dataset_group_arn = response['DatasetGroupArn']

In [ ]:
DATASET_FREQUENCY = "M"
TIMESTAMP_FORMAT = "yyyy-MM-dd"

schema ={
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"demand",
         "AttributeType":"float"
      },
   ]
}

response = forecast.create_dataset(
    Domain = "RETAIL",
    DatasetType = 'TARGET_TIME_SERIES',
    DatasetName = project + "_tts",
    DataFrequency = DATASET_FREQUENCY, 
    Schema = schema
)

tts_dataset_arn = response['DatasetArn']

In [ ]:
schema ={
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"price",
         "AttributeType":"float"
      },
   ]
}

response = forecast.create_dataset(
    Domain = "RETAIL",
    DatasetType = 'RELATED_TIME_SERIES',
    DatasetName = project + "_rts",
    DataFrequency = DATASET_FREQUENCY, 
    Schema = schema
)

rts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.update_dataset_group( 
    DatasetGroupArn = dataset_group_arn, 
    DatasetArns = [
        tts_dataset_arn,
        rts_dataset_arn,
    ]
)

## Import the target time series data, and related time series data

In [ ]:
response = forecast.create_dataset_import_job(
    DatasetImportJobName = project + "_tts_import",
    DatasetArn = tts_dataset_arn,
    DataSource = {
        "S3Config" : {
            "Path" : s3_data_path_tts,
            "RoleArn" : role_arn
        }
    },
    TimestampFormat = TIMESTAMP_FORMAT
)

tts_dataset_import_job_arn = response['DatasetImportJobArn']

In [ ]:
response = forecast.create_dataset_import_job(
    DatasetImportJobName = project + "_rts_import1",
    DatasetArn = rts_dataset_arn,
    DataSource = {
        "S3Config" : {
            "Path" : s3_data_path_rts,
            "RoleArn" : role_arn
        }
    },
    TimestampFormat = TIMESTAMP_FORMAT
)

rts_dataset_import_job_arn = response['DatasetImportJobArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job( DatasetImportJobArn = tts_dataset_import_job_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job( DatasetImportJobArn = rts_dataset_import_job_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

## Create the first Predictor

Creating the 1st Predictor using futurefill option "min".

In [ ]:
# a function to define parameters for create_predictor(), with specified futurefill option.
def define_predictor_params( name, futurefill ):
    return {
        "PredictorName" : name,
        "AlgorithmArn" : "arn:aws:forecast:::algorithm/Deep_AR_Plus",
        "ForecastHorizon" : 3,
        "PerformAutoML" : False,
        "PerformHPO" : False,
        "EvaluationParameters" : {
            "NumberOfBacktestWindows" : 1, 
            "BackTestWindowOffset" : 3
        }, 
        "InputDataConfig" : {
            "DatasetGroupArn" : dataset_group_arn
        },
        "FeaturizationConfig" : {
            "ForecastFrequency" : DATASET_FREQUENCY, 
            "Featurizations" : [
                {
                    # for Target time series
                    "AttributeName" : "demand", 
                    "FeaturizationPipeline" : [
                        {
                            "FeaturizationMethodName": "filling", 
                            "FeaturizationMethodParameters": {
                                "frontfill": "none", 
                                "middlefill": "mean", 
                                "backfill": "mean"
                            }
                        }
                    ]
                },
                {
                    # for Related time series
                    "AttributeName" : "price", 
                    "FeaturizationPipeline" : [
                        {
                            "FeaturizationMethodName": "filling", 
                            "FeaturizationMethodParameters": {
                                "middlefill": "mean", 
                                "backfill": "mean",
                                "futurefill": futurefill,
                            }
                        }
                    ]
                }
            ]
        }
    }


In [ ]:
# for the 1st Predictor, we use "min" for futurefill option
predictor_params = define_predictor_params( project + "_predictor1", futurefill = "min" )

response = forecast.create_predictor( **predictor_params )
predictor1_arn = response['PredictorArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor( PredictorArn = predictor1_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
forecast.get_accuracy_metrics( PredictorArn = predictor1_arn )

## Create Forecast from the first Predictor

In [ ]:
response = forecast.create_forecast(
    ForecastName = project + "_forecast1",
    PredictorArn = predictor1_arn
)

forecast1_arn = response['ForecastArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast( ForecastArn = forecast1_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

## Create 2nd Predictor with different futurefill option

Creating the 2nd Predictor using futurefill option "max".

In [ ]:
# for the 2nd Predictor, we use "max" for futurefill option
predictor_params = define_predictor_params( project + "_predictor2", futurefill = "max" )

response = forecast.create_predictor( **predictor_params )
predictor2_arn = response['PredictorArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor( PredictorArn = predictor2_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
forecast.get_accuracy_metrics( PredictorArn = predictor2_arn )

## Create Forecast from the 2nd predictor

In [ ]:
response = forecast.create_forecast(
    ForecastName = project + "_forecast2",
    PredictorArn = predictor2_arn
)

forecast2_arn = response['ForecastArn']

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast( ForecastArn = forecast2_arn )['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

## Query forecasts, visualize and compare

So far we got two Forecasts for different futurefill options (min vs max). Let's get the forecasted product demands, visualize, and compare.

In [ ]:
training_data_period = ( df_tts["timestamp"].min(), df_tts["timestamp"].max() )

def plot_compare( item_id ):

    plt.figure(figsize=(12, 6))
    plt.title(item_id)
    
    df_item_actual = df_tts[ df_tts["item_id"]==item_id ]
    plt.plot( pd.to_datetime(df_item_actual["timestamp"]), df_item_actual["demand"], label="actual", color=(1,0,0) )

    def plot_forecast( single_item_forecast, label, color, hatch ):

        x = []
        y_p10 = []
        y_p50 = []
        y_p90 = []

        # visually connect last actual value with forecasts
        df_connect = df_item_actual[ df_item_actual["timestamp"]==training_data_period[1] ].reset_index(drop=True)
        x.append( datetime.datetime.strptime( df_connect.at[ 0, "timestamp" ], "%Y-%m-%d" ) )
        y_p10.append( df_connect.at[0,"demand"] )
        y_p50.append( df_connect.at[0,"demand"] )
        y_p90.append( df_connect.at[0,"demand"] )

        for p10, p50, p90 in zip( single_item_forecast["p10"], single_item_forecast["p50"], single_item_forecast["p90"] ):

            date = datetime.datetime.strptime(p50["Timestamp"],"%Y-%m-%dT00:00:00").date()
            x.append(date)

            y_p10.append(p10["Value"])
            y_p50.append(p50["Value"])
            y_p90.append(p90["Value"])

        plt.plot( x, y_p50, label="%s p50" % label, color=color )
        plt.fill_between( x, y_p10, y_p90, label="%s p10-p90" % label, color=color, alpha=0.2, hatch=hatch )

    def plot_price( single_item_price, label, color ):
        
        x = []
        y = []
        
        for timestamp, price in zip( single_item_price["timestamp"], single_item_price["price"] ):
            date = datetime.datetime.strptime(timestamp,"%Y-%m-%d").date()
            x.append(date)
            y.append(price)

        plt.plot( x, y, label=label, color=color, linestyle=":" )
            
    response = forecastquery.query_forecast(
        ForecastArn = forecast1_arn,
        Filters = { "item_id" : item_id }
    )
    plot_forecast( response["Forecast"]["Predictions"], "forecast1", (0,0,1), "+" )

    response = forecastquery.query_forecast(
        ForecastArn = forecast2_arn,
        Filters = { "item_id" : item_id }
    )
    plot_forecast( response["Forecast"]["Predictions"], "forecast2", (1,0,1), "x" )

    bottom, top = plt.ylim()
    plt.ylim((-top*0.03, top*1.03))

    plt.legend( loc='lower left' )


In [ ]:
for item_id in [ "item_010", "item_011", "item_012" ]:
    plot_compare(item_id)


## Resource cleanup

In [ ]:
# Delete forecasts
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast1_arn))
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast2_arn))

In [ ]:
# Delete predictor
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor1_arn))
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor2_arn))

In [ ]:
# Delete dataset import jobs
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn = tts_dataset_import_job_arn))
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn = rts_dataset_import_job_arn))

In [ ]:
# Delete datasets
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn = tts_dataset_arn))
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn = rts_dataset_arn))

In [ ]:
# Delete dataset group
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn = dataset_group_arn))

In [ ]:
# Delete IAM role
util.delete_iam_role( role_name )